In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Softmax, MaxPooling2D, BatchNormalization, ELU, Dropout
from tensorflow.keras.optimizers import Adam
from keras import regularizers
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
tf.__version__

'2.13.0'

# Data Pre-Processing

Open **kyu_train.csv** file and split the games into a list.
Every row of csv: `KL0000000001,B,B[pq],W[dd],B[dp],W[pd],B[jc],...`.

Columns are:

    1. KL0000000001: Game ID
    2. B: Player's color
    3-... : Moves
    
We cropped only the moves to game list as:

In [4]:
df = open('/content/drive/Shareddrives/AICUP圍棋Dataset/kyu_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]

Create a dictionary to convert the coordinates from characters to numbers

In [5]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 4 dimensional feature map to represent the data as below:
 1. Positions of black stones: mark them as 1 and the rest of the table as 0
 2. Positions of white stones: mark them as 1 and the rest of the table as 0
 3. Empty areas of the table: mark the empty areas as 1 and occupied areas as 0
 4. The last move in the table: mark the position of the last move as 1 and the rest as 0

Target value is a number between 0-361(19\*19). Later this will be one-hot encoded.

In [6]:
def prepare_input(moves):
    x = np.zeros((19,19,6))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,3] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

In [7]:
# Check how many samples can be obtained
n_games = 0
n_moves = 0
for game in games:
    n_games += 1
    moves_list = game.split(',')
    for move in moves_list:
        n_moves += 1
print(f"Total Games: {n_games}, Total Moves: {n_moves}")

Total Games: 118500, Total Moves: 27135638


The code below is run for baseline model only by using only the first 500 games from the dataset. You might need to create a data generator to use complete dataset. Otherwise your RAM might not enough to store all (If you run the code on free version of Google Colab, it will crash above 500 game samples).

In [8]:
import random

games = random.sample(games,500)
x = []
y = []
for game in games:
    moves_list = game.split(',')
    for count, move in enumerate(moves_list):
        x.append(prepare_input(moves_list[:count]))
        y.append(prepare_label(moves_list[count]))
x = np.array(x)
y = np.array(y)
print(x)

[[[[0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   ...
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]]

  [[0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   ...
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]]

  [[0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   ...
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]]

  ...

  [[0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   ...
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]]

  [[0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   ...
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]]

  [[0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   ...
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]]]


 [[[0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0.]
   ...
   [0. 0. 1. 0. 0. 0.]
   [0. 0. 

In [ ]:
x.shape

(138168, 19, 19, 5)

In [ ]:
y.shape

(138168,)

In [9]:
y_one_hot = tf.one_hot(y, depth=19*19)

Dataset splitting: 90% Training, 10% validation

In [10]:
x_train, x_val, y_train, y_val = train_test_split(x, y_one_hot.numpy(), test_size=0.10)

# Training

### Simple DCNN Model:

In [11]:
def create_model():
    inputs = Input(shape=(19, 19, 6))
    outputs = Conv2D(kernel_size=16, filters=4, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(inputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=8, filters=4, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=8, filters=2, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=4, filters=2, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=4, filters=1, padding='same', activation='relu',kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)

    outputs = Flatten()(outputs)
    outputs = Softmax()(outputs)

    model = Model(inputs, outputs)

    opt = Adam(learning_rate=0.005)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [12]:
model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 19, 19, 6)]       0         
                                                                 
 conv2d (Conv2D)             (None, 19, 19, 4)         6148      
                                                                 
 batch_normalization (Batch  (None, 19, 19, 4)         16        
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 19, 19, 4)         1028      
                                                                 
 batch_normalization_1 (Bat  (None, 19, 19, 4)         16        
 chNormalization)                                                
                                                                 
 conv2d_2 (Conv2D)           (None, 19, 19, 2)         514   

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

In [13]:
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 256,
    epochs = 30,
    validation_data=(x_val, y_val),
)

Epoch 1/30
401/401 [==============================] - 18s 13ms/step - loss: 5.4271 - accuracy: 0.0213 - val_loss: 4.8998 - val_accuracy: 0.0449
Epoch 2/30
401/401 [==============================] - 5s 12ms/step - loss: 4.7170 - accuracy: 0.0537 - val_loss: 4.8206 - val_accuracy: 0.0574
Epoch 3/30
401/401 [==============================] - 4s 11ms/step - loss: 4.6574 - accuracy: 0.0633 - val_loss: 4.6401 - val_accuracy: 0.0677
Epoch 4/30
401/401 [==============================] - 4s 11ms/step - loss: 4.6419 - accuracy: 0.0646 - val_loss: 4.6651 - val_accuracy: 0.0654
Epoch 5/30
401/401 [==============================] - 5s 12ms/step - loss: 4.6269 - accuracy: 0.0669 - val_loss: 4.6287 - val_accuracy: 0.0688
Epoch 6/30
401/401 [==============================] - 5s 11ms/step - loss: 4.6251 - accuracy: 0.0681 - val_loss: 4.6863 - val_accuracy: 0.0587
Epoch 7/30
401/401 [==============================] - 4s 11ms/step - loss: 4.6174 - accuracy: 0.0680 - val_loss: 4.6978 - val_accuracy: 0.062

In [14]:
model.save('/content/drive/Shareddrives/AICUP圍棋Dataset/model_kyu_tutorial.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Testing

**PublicUpload.csv** must be in the following form:
```
KTPU0000000001,id,qr,pq,pd,ab
KTPU0000000002,ao,ab,ha,ff,qd
KTPU0000000003,qd,gd,fh,ed,fa
KTPU0000000004,pr,ba,dq,hh,aj
KTPU0000000005,ph,jh,af,df,gj
```

- Column 1: Game ID
- Column 2: Predicted Moves, up to 5 predictions for each game

The code block below is to use **kyu_test_public.csv** to predict and save the results in required form. It generates the best 5 predictions for each sample and convert them to character coordinates.

In [15]:
def number_to_char(number):
    number_1, number_2 = divmod(number, 19)
    return chartonumbers[number_1] + chartonumbers[number_2]

def top_5_preds_with_chars(predictions):
    resulting_preds_numbers = [np.flip(np.argpartition(prediction, -5)[-5:]) for prediction in predictions]
    resulting_preds_chars = np.vectorize(number_to_char)(resulting_preds_numbers)
    return resulting_preds_chars

In [16]:
df = open('/content/drive/Shareddrives/AICUP圍棋Dataset/kyu_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input(moves_list))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_chars = top_5_preds_with_chars(predictions)

# Save results to PublicUpload.csv
with open('/content/drive/Shareddrives/AICUP圍棋Dataset/PublicUpload.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

355/355 [==============================] - 1s 2ms/step


# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!

In [17]:
print(prediction_chars)

[['kb' 'ld' 'lc' 'kd' 'jb']
 ['qn' 'om' 'qm' 'pn' 'qo']
 ['bm' 'bl' 'cl' 'cn' 'dm']
 ...
 ['ck' 'cj' 'ci' 'bk' 'dl']
 ['oj' 'nj' 'oi' 'mh' 'nh']
 ['cb' 'dc' 'cd' 'be' 'eb']]
